In [12]:
import pandas as pd
import numpy as np
import re  # numeric extraction
from datetime import datetime # date alignment
import requests # make request to Taxonomic Resolution service "http://tnrs.iplantcollaborative.org/api.html"
from requests_futures.sessions import FuturesSession # used to speed up ITIS API calls (by doing multiple at once)
import json # decode that request

## Step 1: correct misaligned data

- A misaligned row is one in which the columns are all shifted left or right.
    - This only identifies them by checking a for predictable conditions in specific columns.
- This step requires the user to:
    - enter the file name to clean as "inputDataName"
    - manually correct the misaligned rows using a spreadsheet editor program
    - Save the edits as a .csv file and enter that file name

In [13]:
# Enter the name of the input file following the " = " below using quotations and the extension (ie: "records.csv")
inputDataName = 'ICMS DATA UT Chattanooga Citizen Science Digitization.csv'


######################################################################################

# first load in the file as a dataframe 'df'
df = pd.read_csv(inputDataName, dtype = 'str')

def genProblemDF(df):
    ''' This function accepts a dataframe with ICMS headders 
        and returns any potentially misaligned rows'''

    problemDF = df[~(df['Kingdom'].str.upper() == 'PLANTAE') |
               ~(df['Family'].str.contains('ACEAE', na=False, case=False)) |
               ~(df['Object Status'].str.contains('STORAGE', na=False, case=False)) |
               ~(df['Status Date'].str.isnumeric().notna()) |
               ~(df['TSN'].str.isnumeric().notna()) ].copy()

    # replace empty values with an empty string
    problemDF.fillna('', inplace=True)
    
    return problemDF
problemDF = genProblemDF(df)
    
# count how many problem records were identified.
problemCount = problemDF.shape[0]

# if there are issues, print a warning and output a report.
if problemCount > 0:
    problemDF.to_csv('Misaligned Rows {}'.format(inputDataName), encoding = 'utf-8', index=False)
    print('WARNING! Identified {} Misaligned rows. A Report was generated.'.format(problemCount))

# if there are no problems, more along
else:
    print('No misalignment identified! You can skip the rest of this step.')

WARNING! Identified 12 Misaligned rows. A Report was generated.


#### Now manually fix the misaligned rows
- open the "Misaligned Rows .... .csv" and make the necessary corrections manually. Most of these errors where from a researcher attempting to use multiple columns for names instead of comma seperation shifting all the columns over.

- Save the corrections in the same folder

- Enter the file name for the saved corrections with quotations and the extension (ie: 'alignment corrections.csv')

In [14]:
# Enter the file name for the saved corrections with quotations and the extension (ie: 'alignment corrections.csv')
correctedFileName = 'Corrected Misaligned Rows.csv'

######################################################################################


corrections = pd.read_csv(correctedFileName, dtype = 'str')

correctionProblemCount = genProblemDF(corrections).shape[0]

if correctionProblemCount > 0:
    print('{} Misalignment(s) still in "{}"'.format(correctionProblemCount, correctedFileName))
    print('These must be fixed before moving on.')
    print('You can correct the remaining misalignments in "{}" instead of starting over'.format(correctedFileName))

else:
    df = pd.concat([corrections,df]).drop_duplicates(['Catalog #'],keep='first')
    dfName = 'Cleaned {}'.format(inputDataName)
    df.to_csv(dfName, encoding = 'utf-8', index=False)
    print('No problems found, saving the cleaned records as "{}"'.format(dfName))

No problems found, saving the cleaned records as "Cleaned ICMS DATA UT Chattanooga Citizen Science Digitization.csv"


## Step 2: retrieve the TSN numbers

- Uses ITIS's "getITISTermsFromScientificName" API service
    - https://www.itis.gov/ws_searchApiDescription.html#getItisTermsfmSciName

- This step uses the existing 'df' made after correctly applying corrections from step 1

- This step requires the user to 


In [15]:
def get_JSON(row):
    """expects row data delivered as series returns a TSN from itis"""
    potCols = ['Sci. Name:Genus','Sci. Name:Species','Sci. Name:Subspecies','Sci. Name:Variety'] # group of potentitally useful column names

    # ITIS Expects 'ssp.' or 'var.' as part of the query for infraspecific taxa.
    if (row['Sci. Name:Subspecies'] != '') and ('ssp.' not in row['Sci. Name:Subspecies']):
        row['Sci. Name:Subspecies'] = 'ssp. ' + row['Sci. Name:Subspecies']
        
    if (row['Sci. Name:Variety'] != '') and ('var.' not in row['Sci. Name:Variety']):
        row['Sci. Name:Variety'] = 'var. ' + row['Sci. Name:Variety']
        
    potCols = [row[x].rstrip() for x in potCols] # convert column names to values at those columns
    colList = [x for x in potCols if not (pd.isnull(x) or x == '')] # only keep stuff which is not empty

    queryName = ' '.join(colList)
    queryName = queryName.replace(' ', '%20')
    url = 'https://www.itis.gov/ITISWebService/jsonservice/getITISTermsFromScientificName?srchKey={}'.format(queryName)
    
    session = FuturesSession()
    result = session.get(url=url).result().json()
    return result

def parse_TSN(result):

    try:
        TSN = result.get('itisTerms')[0].get('tsn')
    except AttributeError:
        # if the ITIS query returns with no results it fills in a blank.
        # An alternative here would be to query again based on the first word (presumably a genus)
        # and fill in the Genus TSN number.
        # Another option would be to run the name through iPlant API to check for typos.
        TSN = ''

    return TSN


df = df.fillna('')

df['new_TSN'] = df.apply(get_JSON, axis=1).apply(parse_TSN)
    
# Count how many we found and report it.
filledTSN = df[(df['TSN'] == '') & (df['new_TSN'] != '')].shape[0]
print('Retrieved {} previously empty TSN numbers from a total of {} records'.format(filledTSN, df.shape[0]))

# Identify every instance in which the existing TSN value did not match the retrieved one.
problemDF = df[(df['TSN'] != df['new_TSN']) & (df['TSN'] != '')]
problemDF = problemDF[['Catalog #','Family', 'Sci. Name', 'Sci. Name:Genus','Sci. Name:Species','Sci. Name:Subspecies','Sci. Name:Variety','TSN','new_TSN']]

# count how many problems were identified.
problemCount = problemDF.shape[0]

# if there are issues, print a warning and output a report.
if problemCount > 0:
    problemDF.to_csv('MisMatched TSN {}'.format(inputDataName), encoding = 'utf-8', index=False)
    print('WARNING! Identified {} MisMatched TSNs. A Report was generated.'.format(problemCount))
    print('')

# if there are no problems, more along
else:
    print('ALL TSNs Matched! Nice work.')

Retrieved 0 previously empty TSN numbers from a total of 952 records
WARNING! Identified 399 MisMatched TSNs. A Report was generated.



In [24]:
# temp only here to avoid re-running api for testing.
#df.to_csv('postAPI.csv', encoding = 'utf-8')

#### Now manually fix the mismatched TSNs

- open the "MisMatched TSN .... .csv" and decide which TSN column to keep either the original 'TSN' or the 'new_TSN'.
    - All the values from the column you choose will be added to the rows which disagree
    - If you want to keep a mix of both, you can manually change whichever column you plan to pick.

- Save the corrections in the same folder

- Enter the file name for the saved corrections with quotations and the extension (ie: 'Corrected TSN.csv')

- Select which column you plan to keep

In [23]:
# enter the file name for the TSN Corrections file after the equals sign, using quotes
correctedFileName = 'Corrected TSN.csv'

# enter the name of the column to keep using quotes (case sensative)
# the options are the initial 'TSN' or the newly retrieved 'new_TSN'
#columnToKeep = 'TSN'
columnToKeep = 'new_TSN'
######################################################################################


corrections = pd.read_csv(correctedFileName, dtype = 'str').fillna('')

try:
    if columnToKeep == 'TSN':
        # if you choose to keep the initial TSN values, copy them over 'new_TSN.'
        corrections['new_TSN'] = corrections[columnToKeep]
        
    # insert the corrections as 'TSN'
    df = pd.concat([corrections,df], sort=False).drop_duplicates(['Catalog #'],keep='first')
    
    # When it is not empty, copy the 'new_TSN' column into 'TSN'
    #df.loc[df['new_TSN']]
    
    def fillTSN(row):
        row = row.fillna('')
        if row['new_TSN'] != '':
            result = row['new_TSN']
        else:
            result = row['TSN']
            
        return result
    
    df['TSN'] = df.apply(fillTSN, axis=1)
    
    # df.loc[df['new_TSN'] != '','TSN'] = df.loc[df['new_TSN'] != '','new_TSN']
    
    # drop 'new_TSN' from the dataframe
    df = df.drop(columns=['new_TSN'])
    
    dfName = 'Cleaned {}'.format(inputDataName)
    df.to_csv(dfName, encoding = 'utf-8', index=False)
    print('Saving the cleaned records as "{}"'.format(dfName))

except KeyError as e: 
    print(e)
    print('Process halted! {} does not appear to be a column name in {}'.format(columnToKeep, correctedFileName))
    print('Check the columnToKeep name and try again')

Saving the cleaned records as "Cleaned ICMS DATA UT Chattanooga Citizen Science Digitization.csv"
